In [2]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as sp_linalg

In [3]:
def generate_mask(n1, n2, p):
    """
    Generate a mask with probability p for each entry
    :param int n1: number of rows
    :param int n2: number of columns
    :param float p: probability of observing an entry
    """
    omega = np.round(0.5 * (np.random.random((n1, n2)) + p))
    return omega


In [69]:
def generate_ort(p, r):
    E = np.random.randn(p, r)
    E = 1./10 * np.linalg.qr(E)[0]
    return E

In [70]:
# Start parametrs
p = 50
q = 90
r = 5 
prob = 2 * r**2 / (p * q)
K = 10
template = generate_mask(p, q, prob)


In [71]:
#init X, Y, templates
X_0 = np.random.rand(p, q)
eps = np.random.normal(0, 1, (p, q))
omega = generate_mask(p, q, prob)
X = X_0 * omega 
Y = (X_0 + eps) * omega
templates = [generate_mask(p, q, prob) for i in range(K)]

In [131]:
def RGD(Y, X, r, K, templates, reg=0.05, mu_0=0.5, mu_e=0.5, mu_g=0.5):
    p,q = X.shape
    
    #init U and V
    U = generate_ort(p, r)
    V = generate_ort(q, r)
    
    values = np.random.rand(r) 
    L = np.diag(np.sort(values))

    #init E_u and E_v
    E_u = generate_ort(p, r)
    E_v = generate_ort(q, r)

    #init G
    values = np.random.rand(r) 
    sort_values = np.sort(values)[::-1]
    G = np.diag(sort_values)


    iter_num = 0
    max_iter = 30
    I_r = np.eye(r)

    while iter_num < max_iter:
        iter_num += 1

        nabla_U = 0
        nabla_V = 0
        nabla_l = 0

        for k in range(K):
            nabla_U += (np.sum(X * templates[k]) - np.sum(templates[k] * (U @ L @ V.T))) * templates[k] @ V @ L
            nabla_V += (np.sum(X * templates[k]) - np.sum(templates[k] * (U @ L @ V.T))) * templates[k].T @ U @ L
            nabla_l += (np.sum(X * templates[k]) - np.sum(templates[k] * (U @ L @ V.T))) * U.T @ templates[k] @ V

        nabla_U += - mu_0 * U @ (U.T @ U - I_r) - mu_e * (U - E_u)
        nabla_V += - mu_0 * V @ (V.T @ V - I_r) - mu_e * (V - E_v)
        nabla_l += - mu_g * G @ L

        U = U - reg * nabla_U 
        V = V - reg * nabla_V
        L = L - reg * nabla_l


    return U @ L @ V.T
    

In [132]:
X_new = RGD(Y, X, r, K, templates)

In [133]:
true_error = np.linalg.norm(X_new - X_0, ord='fro') / np.linalg.norm(X_0)
observed_error = np.linalg.norm((X_new - X_0) * omega, ord='fro') / np.linalg.norm(X_0)
print('true error: {}, observed error: {}'.format(true_error, observed_error))

true error: 1.001079359776784, observed error: 0.10912085721481063


In [143]:
#Из-за того, что определяю все матрицы рандомом, то ответ от случая к случаю разный

In [140]:
X_new_2 = RGD(Y, X, r, K, templates)

In [142]:
true_error = np.linalg.norm(X_new_2 - X_0, ord='fro') / np.linalg.norm(X_0)
observed_error = np.linalg.norm((X_new_2 - X_0) * omega, ord='fro') / np.linalg.norm(X_0)
print('true error: {}, observed error: {}'.format(true_error, observed_error))

true error: 9.027519097523414e+31, observed error: 5.701657898010583e+30
